# Ранжирование

Будем ранжировать разные данные с помощью XGBoost. <br>
В XGBoost используется алгоритм LambdaMART, который осуществляет pairwise ранжирование.

## Пример

Взят [отсюда](https://github.com/dmlc/xgboost/blob/master/demo/rank/rank_sklearn.py)

In [1]:
import numpy as np
import xgboost as xgb
from sklearn.datasets import load_svmlight_file
from sklearn.metrics import ndcg_score

Данные: датасет LETOR 4.0 MQ2008 - бенчмарк для ранжирования.

Данные представлены так:
* один объект - это запрос, один документ к нему (набор каких-то признаков) и одна метка релевантности (target)
* соответственно, для одного и того же запроса может быть несколько объектов
* информация, что, например, какие-то пять объектов относятся к одному запросу, содержится в отдельной структуре "groups" и передается в обучение

Читаем объекты и таргеты:

In [2]:
x_train, y_train = load_svmlight_file("data/mq2008.train")
x_valid, y_valid = load_svmlight_file("data/mq2008.vali")
x_test, y_test = load_svmlight_file("data/mq2008.test")

Посмотрим на данные:
* в обучении 9630 объектов
* 46 признаков
* релевантность оценивается по трехбалльной шкале

In [3]:
print(x_train.shape)
x_train[0].todense()

(9630, 46)


matrix([[0.007477, 0.      , 1.      , 0.      , 0.00747 , 0.      ,
         0.      , 0.      , 0.      , 0.      , 0.471076, 0.      ,
         1.      , 0.      , 0.477541, 0.00512 , 0.      , 0.571429,
         0.      , 0.004806, 0.768561, 0.727734, 0.716277, 0.582061,
         0.      , 0.      , 0.      , 0.      , 0.780495, 0.962382,
         0.999274, 0.961524, 0.      , 0.      , 0.      , 0.      ,
         0.797056, 0.697327, 0.721953, 0.582568, 0.      , 0.      ,
         0.      , 0.      , 0.      , 0.007042]])

In [4]:
set(y_train)

{0.0, 1.0, 2.0}

Читаем информацию о группах:

In [5]:
group_train = []
with open("data/mq2008.train.group", "r") as f:
    data = f.readlines()
    for line in data:
        group_train.append(int(line.split("\n")[0]))

group_valid = []
with open("data/mq2008.vali.group", "r") as f:
    data = f.readlines()
    for line in data:
        group_valid.append(int(line.split("\n")[0]))

group_test = []
with open("data/mq2008.test.group", "r") as f:
    data = f.readlines()
    for line in data:
        group_test.append(int(line.split("\n")[0]))

Как устроена информация о группах:
* количество групп отражает информацию о количестве запросов
* каждое число обозначает количество последовательных объектов, которые в эту группу объединяются
* из предыдущего пункта следует, что в X объекты нельзя перемешивать
* если просуммировать все числа в списке групп, получим число объектов из X

Для чего нужны группы? <br>
Для того, чтобы в обучении не сравнивать доки из разных групп (разных запросов) между собой.

In [6]:
print(len(group_train), sum(group_train))
group_train[:10]

471 9630


[8, 8, 8, 8, 8, 16, 8, 118, 16, 8]

Обучаем модель. <br>
С помощью `eval_set` можем контролировать обучение, но это необязательный параметр, можно обучить и без валидации. <br>
В параметре `objective` можно задать три опции: `rank:ndcg`, `rank:pairwise`, `rank:map`. `ndcg` и `map` регулияруют попарный лосс с помощью подсчета соответствующих метрик.

In [7]:
params = {'objective': 'rank:ndcg', 'learning_rate': 0.1,
          'gamma': 1.0, 'min_child_weight': 0.1,
          'max_depth': 6, 'n_estimators': 4}

model = xgb.sklearn.XGBRanker(**params)
model.fit(x_train, y_train, group_train, verbose=True,
          eval_set=[(x_valid, y_valid)], eval_group=[group_valid])

[0]	eval_0-map:0.71552
[1]	eval_0-map:0.72606
[2]	eval_0-map:0.72795
[3]	eval_0-map:0.73352


XGBRanker(base_score=0.5, booster='gbtree', colsample_bylevel=1,
          colsample_bynode=1, colsample_bytree=1, gamma=1.0, gpu_id=-1,
          importance_type='gain', interaction_constraints='', learning_rate=0.1,
          max_delta_step=0, max_depth=6, min_child_weight=0.1, missing=nan,
          monotone_constraints='()', n_estimators=4, n_jobs=0,
          num_parallel_tree=1, objective='rank:ndcg', random_state=0,
          reg_alpha=0, reg_lambda=1, scale_pos_weight=None, subsample=1,
          tree_method='exact', validate_parameters=1, verbosity=None)

Получим предсказание на тестовом сете:

In [8]:
pred = model.predict(x_test)

Посчитаем качество:

In [9]:
start_idx = 0
grouped_pred = []
grouped_target = []

for group_n in group_test:
    grouped_pred.append(pred[start_idx:start_idx+group_n])
    grouped_target.append(y_test[start_idx:start_idx+group_n])
    start_idx += group_n

In [10]:
np.mean([
    ndcg_score([grouped_target[i]], [grouped_pred[i]])
    for i in range(len(grouped_target))
])

0.5052327963105946

# Семинар и дз
Сделать и улучшить любую ML-модель на ваших проектных данных (просто клф, бленд, ранжирование, что-то что вы придумали сами...), используя любые признаки, какие захотите. Оцениваться будут:
* факт выполнения задания :)
* корректность кода (чтобы код не падал) и отсутствие логических ошибок (e.g. затестили на трейне)
* итеративность улучшения (например взяли один сет признаков, показали качество; потом добавили / подкрутили / использовали другую модель, показали качество...)
* креативность признаков
* аккуратность ноутбука

Дедлайн: 15 октября